In [ ]:
!pip install transformers datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 48.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 2.2 MB/s eta 0:00:00


In [ ]:

from datasets import load_dataset
data=load_dataset('conll2003')


Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

Dataset conll2003 downloaded and prepared to /root/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [ ]:
data['train']

Dataset({
    features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
    num_rows: 14041
})

In [ ]:
data['train'].features['tokens']

Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)

In [ ]:
data['train'].features['ner_tags']


Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)

In [ ]:
data['train'].features['ner_tags'].feature.names


['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [ ]:
label_names=data['train'].features['ner_tags'].feature.names
label_names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [ ]:
from transformers import AutoTokenizer
checkpoint='distilbert-base-cased'
tokenizer=AutoTokenizer.from_pretrained(checkpoint)


In [ ]:
idx=0
t=tokenizer(data['train'][idx]['tokens'],is_split_into_words=True)
t

{'input_ids': [101, 7270, 22961, 1528, 1840, 1106, 21423, 1418, 2495, 12913, 119, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:

type(t)
t.tokens


<bound method BatchEncoding.tokens of {'input_ids': [101, 7270, 22961, 1528, 1840, 1106, 21423, 1418, 2495, 12913, 119, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}>

In [ ]:
begin2inside={
    1:2,
    3:4,
    5:6,
    7:8

}
def align_targets(labels,word_ids):
  aligned_labels=[]
  last_word=None
  for word in word_ids:

    if word is None:
      label=-100
    elif word!=last_word:
      label=labels[word]
    else:
      label=labels[word]
      if label in begin2inside:
        label=begin2inside[label]
    aligned_labels.append(label)
    last_word=word
  return aligned_labels


In [ ]:

#try our functin
labels=data['train'][idx]['ner_tags']
labels


[3, 0, 7, 0, 0, 0, 7, 0, 0]

In [ ]:
word_ids=t.word_ids()
word_ids

[None, 0, 1, 2, 3, 4, 5, 6, 7, 7, 8, None]

In [ ]:
aligned_targets=align_targets(labels,word_ids)
print(aligned_targets)

[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]


In [ ]:
aligned_labels=[label_names[t] if t>=0 else  None for t in aligned_targets ]
aligned_labels

[None, 'B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O', 'O', None]

In [ ]:
for x,y in zip(t.tokens(),aligned_labels):
  print(f'{x}\t{y}')

[CLS]	None
EU	B-ORG
rejects	O
German	B-MISC
call	O
to	O
boycott	O
British	B-MISC
la	O
##mb	O
.	O
[SEP]	None


In [ ]:

#make up fake input just to test it
words=[
    '[CLS]','Ger','man','call','to','boycott','micro','soft','[SEP]']
word_ids=[None,0,0,1,2,3,4,4,None]
labels=[7,0,0,0,3]
aligned_targets=align_targets(labels,word_ids)
aligned_targets

[-100, 7, 8, 0, 0, 0, 3, 4, -100]

In [ ]:
aligned_labels=[label_names[t] if t>=0 else None for t in aligned_targets]
aligned_labels

[None, 'B-MISC', 'I-MISC', 'O', 'O', 'O', 'B-ORG', 'I-ORG', None]

In [ ]:
for x,y in zip(words,aligned_labels):
  print(f'{x}\t{y}')

[CLS]	None
Ger	B-MISC
man	I-MISC
call	O
to	O
boycott	O
micro	B-ORG
soft	I-ORG
[SEP]	None


In [ ]:
def tokenize_fn(batch):
  tokenized_inputs=tokenizer(
      batch['tokens'],
      truncation=True,
      is_split_into_words=True
  )

  labels_batch=batch['ner_tags']#original tags
  aligned_labels_batch=[]

  for i,labels in enumerate(labels_batch):
    word_ids=tokenized_inputs.word_ids(i)
    aligned_labels_batch.append(align_targets(labels,word_ids))

  #recall the target must be stored in any key called labels
  tokenized_inputs['labels']=aligned_labels_batch
  return tokenized_inputs

In [ ]:
data['train'].column_names


['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags']

In [ ]:
tokenized_datasets=data.map(tokenize_fn, batched=True, remove_columns=data['train'].column_names,)
tokenized_datasets

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 3453
    })
})

In [ ]:
len(tokenized_datasets['train'][0].get('input_ids'))

12

In [ ]:
len(tokenized_datasets['train'][0].get('labels'))

12

In [ ]:
from transformers import DataCollatorForTokenClassification
data_collator=DataCollatorForTokenClassification(tokenizer=tokenizer)


In [ ]:
tokenized_datasets['train'][0]


{'input_ids': [101,
  7270,
  22961,
  1528,
  1840,
  1106,
  21423,
  1418,
  2495,
  12913,
  119,
  102],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]}

In [ ]:
tokenized_datasets['train'][0:2]

{'input_ids': [[101,
   7270,
   22961,
   1528,
   1840,
   1106,
   21423,
   1418,
   2495,
   12913,
   119,
   102],
  [101, 1943, 14428, 102]],
 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1]],
 'labels': [[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100], [-100, 1, 2, -100]]}

In [ ]:
[tokenized_datasets['train'][i] for i in range(2)]


[{'input_ids': [101,
   7270,
   22961,
   1528,
   1840,
   1106,
   21423,
   1418,
   2495,
   12913,
   119,
   102],
  'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  'labels': [-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]},
 {'input_ids': [101, 1943, 14428, 102],
  'attention_mask': [1, 1, 1, 1],
  'labels': [-100, 1, 2, -100]}]

In [ ]:
#example
batch=data_collator([tokenized_datasets['train'][i] for i in range(2)])
batch['labels']


tensor([[-100,    3,    0,    7,    0,    0,    0,    7,    0,    0,    0, -100],
        [-100,    1,    2, -100, -100, -100, -100, -100, -100, -100, -100, -100]])

In [ ]:
!pip install seqeval


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16165 sha256=151bd8ee64dfc044d7cb0c27aa34c4d3c02a2e9b4a3d23db1a31408eea351223
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [ ]:

from datasets import load_metric
metric=load_metric('seqeval')

<ipython-input-75-3bdac680cea9>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric=load_metric('seqeval')


In [ ]:
metric.compute(predictions=[[0,0,0]],references=[[0,0,1]])


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 0 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 1 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:4

{'overall_precision': 0.0,
 'overall_recall': 0.0,
 'overall_f1': 0.0,
 'overall_accuracy': 0.6666666666666666}

In [ ]:
metric.compute(predictions=[['A','A','A']],references=[['A','B','A']])


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: A seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


{'overall_precision': 0.0,
 'overall_recall': 0.0,
 'overall_f1': 0.0,
 'overall_accuracy': 0.6666666666666666}

In [ ]:
import numpy as np
def compute_metrics(logits_and_labels):
  logits,labels=logits_and_labels

  #remove -100 from labels and predictions
  #and convert the label_ids to label_names
  str_labels=[
      [label_names[t] for t in label if t!= -100] for label in labels
  ]
  #do the same for predictions whenever true label is -100
  str_preds=[
      [label_names[p] for p, t in zip(pred,targ) if t!= -100] for pred,targ in zip(preds,labels)
  ]
  the_metrics=metric.compute(predictions=str_preds,references=str_labels)
  return{
      'precision':the_metrics['overall_precision'],
      'recall':the_metrics['overall_recall'],
      'f1':the_metrics['overall_f1'],
      'accuracy':the_metrics['overall_accuracy']
  }

In [ ]:
id2label={k:v for k ,v in enumerate(label_names)}
id2label

{0: 'O',
 1: 'B-PER',
 2: 'I-PER',
 3: 'B-ORG',
 4: 'I-ORG',
 5: 'B-LOC',
 6: 'I-LOC',
 7: 'B-MISC',
 8: 'I-MISC'}

In [ ]:
label2id={v:k for k ,v in id2label.items()}
label2id

{'O': 0,
 'B-PER': 1,
 'I-PER': 2,
 'B-ORG': 3,
 'I-ORG': 4,
 'B-LOC': 5,
 'I-LOC': 6,
 'B-MISC': 7,
 'I-MISC': 8}

In [ ]:
from transformers import AutoModelForTokenClassification
model=AutoModelForTokenClassification.from_pretrained(
    checkpoint,
    id2label=id2label,
    label2id=label2id
)

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:


from transformers import TrainingArguments

In [ ]:

!pip install torch
!pip install tansformers
!pip install transformers[torch]
!pip install accelerate -U

ERROR: Could not find a version that satisfies the requirement tansformers (from versions: none)
ERROR: No matching distribution found for tansformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 3.8 MB/s eta 0:00:00


In [ ]:

training_arguments=TrainingArguments(
    'distilbert-finetuned-ner',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01
)

In [84]:
from transformers import Trainer
trainer=Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer

)
trainer.train()


In [ ]:
trainer.save_model('my_ner_saved_model')
from transformers import pipeline
ner=pipeline(
    'token-classification',
    model='my_ner_saved_model',
    aggregation_strategy='simple',
    device=0
)
s='Hari is the hero of movie Bahubali which was shooted in USA'
ner(s)
